In [47]:
import sendgrid 
import os

from dotenv import load_dotenv
from agents import Agent, Runner, trace, function_tool
from IPython.display import Markdown, display

from sendgrid.helpers.mail import Mail, Email, To, Content
from typing import Dict


In [48]:
load_dotenv(override=True)

True

In [49]:
@function_tool
def send_email(subject: str, html_body: str) -> Dict[str, str]:
    """ send and email with the given subject and convert to HTML body """
    print("sending_email...")
    print(f"From: {os.environ.get('EMAIL_FROM')}")
    print(f"To: trungnv184@gmail.com")
    print(f"Subject: {subject}")
    
    try:            
        sg = sendgrid.SendGridAPIClient(api_key=os.environ.get("SENDGRID_API_KEY"))
        from_email = Email(os.environ.get("EMAIL_FROM"))
        to_email = To("trungnv184@gmail.com")
        content = Content("text/html", html_body)
        mail = Mail(from_email, to_email, subject, content) .get() # Remove .get()
        
        print("Attempting to send email...")
        response = sg.client.mail.send.post(request_body=mail)
        print(f"SendGrid Response Status Code: {response.status_code}")
        
        if response.status_code == 202:
            print("Email sent successfully")
            return {"status": "success"}
        else:
            error_msg = f"Email sending failed with status code: {response.status_code}"
            print(error_msg)
            return {"status": "error", "message": error_msg}
            
    except Exception as e:
        error_msg = f"Error sending email: {str(e)}"
        print(error_msg)        
        return {"status": "error", "message": error_msg}

In [50]:
# create the agent instance
agent = Agent(
    name="Andy",
    instructions="""Please help me to create a learning plan for the next 3 months to become a great AI Agent.
    For each week, please create a list of topics to learn and the resources to learn them.
    For each resource please attach the link to the resource.
    Use send_email tool to send the learning plan to the email address trungnv184@gmail.com with html format.""",
    tools=[send_email],
    model="gpt-4o-mini"
)


In [52]:
with trace("Create the learning plan"):
    result = await Runner.run(agent, "Create a learning plan for the next 3 months to become a great AI Agent")
   # print(result.final_output)
    display(Markdown(result.final_output))


sending_email...
From: andynguyenvn2023@gmail.com
To: trungnv184@gmail.com
Subject: 3-Month AI Learning Plan
Attempting to send email...
SendGrid Response Status Code: 202
Email sent successfully


The 3-month learning plan to become a great AI Agent has been successfully sent to your email at trungnv184@gmail.com. If you need any more assistance or adjustments to the plan, feel free to ask!